# 1. Подготовка данных

In [1]:
!pip install implicit==0.4.0

     |████████████████████████████████| 1.1MB 4.8MB/s 
  Created wheel for implicit: filename=implicit-0.4.0-cp36-cp36m-linux_x86_64.whl size=3390326 sha256=1613ba44c04b345648d0b57ceeae4b58b402902e00c7deba175e82840eda66df
  Stored in directory: /root/.cache/pip/wheels/fd/d1/66/84e83191d9f626c521271023e4ef83e634e3b8c154f2c4b2da
Successfully built implicit


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit import als

# Модель второго уровня
from lightgbm import LGBMClassifier
from lightgbm import LGBMRanker
import lightgbm

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

# Написанные нами функции
#from src.metrics import recall_at_k#, precision_at_k
#from src.utils import prefilter_items
#from src.recommenders import MainRecommender

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!cp /content/drive/'My Drive'/retail_train.csv /content
!cp /content/drive/'My Drive'/product.csv /content
!cp /content/drive/'My Drive'/hh_demographic.csv /content
!cp -r /content/drive/'My Drive'/src /content

In [6]:
data = pd.read_csv('retail_train.csv')
item_features = pd.read_csv('product.csv')
user_features = pd.read_csv('hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)


# Важна схема обучения и валидации!
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 
# подобрать размер 2-ого датасета (6 недель) --> learning curve (зависимость метрики recall@k от размера датасета)
val_lvl_1_size_weeks = 6
val_lvl_2_size_weeks = 3

data_train_lvl_1 = data[data['week_no'] < data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]
data_val_lvl_1 = data[(data['week_no'] >= data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)) &
                      (data['week_no'] < data['week_no'].max() - (val_lvl_2_size_weeks))]

data_train_lvl_2 = data_val_lvl_1.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться
data_val_lvl_2 = data[data['week_no'] >= data['week_no'].max() - val_lvl_2_size_weeks]

data_train_lvl_1.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [7]:
print(data_train_lvl_1['item_id'].nunique())
print(data_train_lvl_2['item_id'].nunique())
print(data_val_lvl_2['item_id'].nunique())

83685
27649
24329


In [8]:
def precision_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)

    bought_list = bought_list  # Тут нет [:k] !!

    if k < len(recommended_list):
        recommended_list = recommended_list[:k]

    flags = np.isin(bought_list, recommended_list)

    precision = flags.sum() / len(recommended_list)

    return precision

In [9]:
def money_precision_at_k(recommended_list, bought_list, prices_recommended, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    prices_recommended = np.array(prices_recommended)


    if k < len(recommended_list):
        recommended_list = recommended_list[:k]
        prices_recommended = prices_recommended[:k]

    flags = np.isin(recommended_list, bought_list)
    res = np.dot(flags, prices_recommended) / np.sum(prices_recommended)

    return res

In [10]:
def recall_at_k(recommended_list, bought_list, k=5):
    import numpy as np
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    recommended_list = recommended_list[:k]
    flags = np.isin(bought_list, recommended_list)
    recall = flags.sum() / len(bought_list)

    return recall

# 2. Класс MainRecommender

In [11]:
import pandas as pd
import numpy as np

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight
from implicit.bpr import BayesianPersonalizedRanking

class MainRecommender:
    """Рекоммендации, которые можно получить из ALS
    Input
    -----
    user_item_matrix: pd.DataFrame
        Матрица взаимодействий user-item
    """

    def __init__(self, data, item_features, weighting='bm25_weight', model_type = 'als', n_factors=160, 
                    regularization=0.001, iterations=30, num_threads=32, random_state=None):

        self.random_state = random_state
#----------------------------------------------------------------------------------------------------
        # Топ покупок каждого юзера
        self.top_purchases = data.groupby(['user_id', 'item_id'])['quantity'].count().reset_index()
        self.top_purchases.sort_values('quantity', ascending=False, inplace=True)
        self.top_purchases = self.top_purchases[self.top_purchases['item_id'] != 999999]
#----------------------------------------------------------------------------------------------------
        # Топ покупок по всему датасету
        self.overall_top_purchases = data.groupby('item_id')['quantity'].count().reset_index()
        self.overall_top_purchases.sort_values('quantity', ascending=False, inplace=True)
        self.overall_top_purchases = self.overall_top_purchases[self.overall_top_purchases['item_id'] != 999999]
        self.overall_top_purchases = self.overall_top_purchases.item_id.tolist()

        self.user_item_matrix = self._prepare_matrix(data)  # pd.DataFrame
        self.id_to_itemid, self.id_to_userid, \
            self.itemid_to_id, self.userid_to_id = self._prepare_dicts(self.user_item_matrix)
#----------------------------------------------------------------------------------------------------
        # Параметры взвешивания
        if weighting =='bm25_weight':
          self.user_item_matrix = bm25_weight(self.user_item_matrix.T, K1=80, B=0.6).T
        elif weighting =='tfidf_weight':
          self.user_item_matrix = tfidf_weight(self.user_item_matrix.T).T
        else:
          pass

#----------------------------------------------------------------------------------------------------
        # Обучение модели коллаборативной фильтрации
        self.model = self.fit(self.user_item_matrix, 
                              model_type = model_type,
                              n_factors=n_factors, 
                              regularization=regularization, 
                              iterations=iterations,
                              num_threads=num_threads)
#----------------------------------------------------------------------------------------------------
        # Обучение модели на основе KNN
        self.own_recommender = self.fit_own_recommender(self.user_item_matrix)

#----------------------------------------------------------------------------------------------------
        # Сохранение признаков, извлеченных из als (bpr)
        self.als_user_factors = self.model.user_factors
        self.als_item_factors = self.model.item_factors

#----------------------------------------------------------------------------------------------------
        # Датафрейм, индексами которого являются товары, а столбцами - 
        # средняя цена по входному датасету для каждого товара и sub_commodity_desc для товара
        self.item_info = data.groupby('item_id').agg({'sales_value':'sum','quantity':'sum'})\
                                                                                    .reset_index()
        # mean_price - результат деления суммы всех sales_value для товара на сумму всех quantity
        # для товара, так как товар может за один раз продаться несколько раз
        self.item_info['mean_price'] = self.item_info['sales_value']/\
                                                                  self.item_info['quantity']
        self.item_info = self.item_info.merge\
                (item_features[['item_id', 'sub_commodity_desc']], how='left', on='item_id')

        self.item_info.index = self.item_info['item_id']
        self.item_info.drop(columns=['sales_value','quantity','item_id'], inplace=True)

#----------------------------------------------------------------------------------------------------
        # Датафрейм, индексами которого являются пользователи, а столбцом - список всех
        # уникальных товаров, которые купил пользователь
        self.users_purchases = data.groupby('user_id')['item_id'].unique().reset_index()
        self.users_purchases.index = self.users_purchases['user_id']
        self.users_purchases.drop(columns = 'user_id', inplace=True)

#----------------------------------------------------------------------------------------------------
        # Датафрейм, индексами которого являются пользователи, а столбцом - 
        # топ товаров, наиболее часто покупаемых пользователем (стоимость товаров выше mprice_theashold)
        mprice_theashold = 4
        data_reduce = data[data['price'] > 4]
        data_reduce = data_reduce[data_reduce['item_id'] != 999999]

        # Датафрейм "user_id - общее количество покупок пользователем товаров стоимостью выше mprice_theashold"
        user_purchases = data_reduce.groupby(['user_id'])['quantity'].sum().reset_index()
        user_purchases.rename(columns={'quantity': 'user_purchases'}, inplace=True)

        # Датафрейм "user_id - item_id - количество покупок пользователем каждого товара стоимостью выше mprice_theashold"
        user_item_purchases = data_reduce.groupby(['user_id', 'item_id'])['quantity'].sum().reset_index()

        # Собираем все в один датасет
        user_item_purchases =  user_item_purchases.merge(user_purchases, on=['user_id'], how='left')

        # Вычисляем частотность покупок для каждого пользователя каждого товара
        user_item_purchases['item_freq'] = user_item_purchases['quantity'] / user_item_purchases['user_purchases']
        user_item_purchases.sort_values(by=['user_id', 'item_freq'], ascending=False, inplace=True)
        #print(user_item_purchases)
        self.top_items_for_user = user_item_purchases.groupby('user_id')['item_id'].unique().reset_index()
        self.top_items_for_user.index = self.top_items_for_user.user_id
        self.top_items_for_user.drop(columns = ['user_id'], inplace=True)

#----------------------------------------------------------------------------------------------------
        # Топ товаров, которые купило наибольшее число уникальных покупателей (стоимость товаров выше mprice_theashold)

        all_users_quantity = data_reduce['user_id'].nunique()

        # Вычисляем количество уникальных покупателей, которые купили данный товар
        all_top_items = data_reduce.groupby(['item_id', 'price'])['user_id'].nunique().reset_index()
        all_top_items.rename(columns={'user_id': 'users_quantity'}, inplace=True)

        # Вычисляем какая доля от всех пользователей купила данный товар
        all_top_items['users_share'] = all_top_items['users_quantity'] / all_users_quantity
        all_top_items  = all_top_items[all_top_items['users_share'] >= np.quantile(all_top_items['users_share'], 0.75)]  
        all_top_items.sort_values('users_share', ascending=False, inplace=True)
        self.all_top_items = all_top_items.item_id.values.tolist()

#----------------------------------------------------------------------------------------------------
        # Датафрейм, индексами которого являются пользователи, а столбцом - 
        # топ товаров, наиболее часто встречаемых в карзинах данного пользователя (стоимость товаров выше mprice_theashold)

        user_basket = data_reduce.groupby(['user_id'])['basket_id'].nunique().reset_index()
        user_basket.rename(columns={'basket_id': 'users_baskets_cnt'}, inplace=True)
        user_item_basket = data_reduce.groupby(['user_id', 'item_id'])['basket_id'].nunique().reset_index()
        user_item_basket = user_item_basket.merge(user_basket, on=['user_id'], how='left')
        user_item_basket['item_freq_in_user_basket'] = user_item_basket['basket_id'] / user_item_basket['users_baskets_cnt']
        user_item_basket.sort_values(by=['user_id', 'item_freq_in_user_basket'], ascending=False, inplace=True)
        self.top_items_in_users_basket = user_item_basket.groupby('user_id')['item_id'].unique().reset_index()
        self.top_items_in_users_basket.index = self.top_items_in_users_basket.user_id
        self.top_items_in_users_basket.drop(columns = ['user_id'], inplace=True)

    @staticmethod
    def _prepare_matrix(data):
        """Готовит user-item матрицу"""
        # ТОЖЕ sales_value ПОДЕЛИТЬ НА quantity
        user_item_matrix = pd.pivot_table(data,
                                          index='user_id', columns='item_id',
                                          #values = 'sales_value', #выручка без затух.
                                          #values = 'sales_value(t)', #выручка с затух.
                                          #values = 'quantity(t)', #число продаж с затух.
                                          #values = 'quantity', #число продаж без затух.
                                          values = 'price(t)', #цена товара с затух.
                                          aggfunc='sum',
                                          fill_value=0
                                          )
        

        user_item_matrix = user_item_matrix.astype(float)  # необходимый тип матрицы для implicit
        #user_item_matrix = np.log1p(user_item_matrix) # Не сработало
        return user_item_matrix

    @staticmethod
    def _prepare_dicts(user_item_matrix):
        """Подготавливает вспомогательные словари"""

        userids = user_item_matrix.index.values
        itemids = user_item_matrix.columns.values

        matrix_userids = np.arange(len(userids))
        matrix_itemids = np.arange(len(itemids))

        id_to_itemid = dict(zip(matrix_itemids, itemids))
        id_to_userid = dict(zip(matrix_userids, userids))

        itemid_to_id = dict(zip(itemids, matrix_itemids))
        userid_to_id = dict(zip(userids, matrix_userids))

        return id_to_itemid, id_to_userid, itemid_to_id, userid_to_id

    @staticmethod
    def fit_own_recommender(user_item_matrix):
        """Обучает модель, которая рекомендует товары, среди товаров, купленных юзером"""

        own_recommender = ItemItemRecommender(K=1, num_threads=8)
        own_recommender.fit(csr_matrix(user_item_matrix).T.tocsr())

        return own_recommender

    @staticmethod
    def fit(user_item_matrix, n_factors=160, regularization=0.001, 
            iterations=30, num_threads=32, model_type='als'):
        """Обучает ALS"""
        if model_type=='als':
          model = AlternatingLeastSquares(factors=n_factors,
                                          regularization=regularization,
                                          iterations=iterations,
                                          num_threads=num_threads,
                                          )
        elif model_type=='bpr':
          model = BayesianPersonalizedRanking(factors=n_factors,
                                          regularization=regularization,
                                          iterations=iterations,
                                          num_threads=num_threads,
                                          )
        else:
          pass

        #if self.random_state != None:
        #  np.random.seed(self.random_state)

        model.fit(csr_matrix(user_item_matrix).T.tocsr())

        return model

    def _update_dict(self, user_id):
        """Если появился новыю user / item, то нужно обновить словари"""

        if user_id not in self.userid_to_id.keys():

            max_id = max(list(self.userid_to_id.values()))
            max_id += 1

            self.userid_to_id.update({user_id: max_id})
            self.id_to_userid.update({max_id: user_id})

    def _get_similar_item(self, item_id):
        """Находит товар, похожий на item_id"""
        recs = self.model.similar_items(self.itemid_to_id[item_id], N=2)  # Товар похож на себя -> рекомендуем 2 товара
        top_rec = recs[1][0]  # И берем второй (не товар из аргумента метода)
        return self.id_to_itemid[top_rec]

    def _extend_with_top_popular(self, recommendations, N=5):
        """Если кол-во рекоммендаций < N, то дополняем их топ-популярными"""

        if len(recommendations) < N:
            recommendations.extend(self.overall_top_purchases[:N])
            recommendations = recommendations[:N]

        return recommendations

    def _get_recommendations(self, user, model, N=5):
        """Рекомендации через стардартные библиотеки implicit"""

        self._update_dict(user_id=user)

        # Чтобы гиперпараметры было удобнее подбирать, зафиксируем seed
        if self.random_state != None:
          np.random.seed(self.random_state)

        res = [self.id_to_itemid[rec[0]] for rec in model.recommend(userid=self.userid_to_id[user],
                                        user_items=csr_matrix(self.user_item_matrix).tocsr(),
                                        N=N,
                                        filter_already_liked_items=False,
                                        filter_items=[self.itemid_to_id[999999]])]
                                        #recalculate_user=True)] в bpr не работает

        res = self._extend_with_top_popular(res, N=N)

        assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
        return res

    def get_als_recommendations(self, user, N=5):
        """Рекомендации через стардартные библиотеки implicit"""
        
        # Обработка холодного пользователя
        if user not in self.userid_to_id.keys():
          #self._update_dict(user_id=user)
          return self.overall_top_purchases[:N]
        else:
        #self._update_dict(user_id=user)
          return self._get_recommendations(user, model=self.model, N=N)

    def get_own_recommendations(self, user, N=5):
        """Рекомендуем товары среди тех, которые юзер уже купил"""

        if user not in self.userid_to_id.keys():
          return self.overall_top_purchases[:N]
        else:
          return self._get_recommendations(user, model=self.own_recommender, N=N)
        #self._update_dict(user_id=user)
        

    def get_similar_items_recommendation(self, user, N=5):
        """Рекомендуем товары, похожие на топ-N купленных юзером товаров"""
        if user not in self.userid_to_id.keys():
          return self.overall_top_purchases[:N]
        else:
          top_users_purchases = self.top_purchases[self.top_purchases['user_id'] == user].head(N)

          res = top_users_purchases['item_id'].apply(lambda x: self._get_similar_item(x)).tolist()
          res = self._extend_with_top_popular(res, N=N)

          assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
          return res

    def get_similar_users_recommendation(self, user, N=5):
        """Рекомендуем топ-N товаров, среди купленных похожими юзерами"""
        if user not in self.userid_to_id.keys():
          return self.overall_top_purchases[:N]
        else:
          res = []

          # Находим топ-N похожих пользователей
          similar_users = self.model.similar_users(self.userid_to_id[user], N=N+1)
          similar_users = [rec[0] for rec in similar_users]
          similar_users = similar_users[1:]   # удалим юзера из запроса

          for user in similar_users:
            res.extend(self.get_own_recommendations(user, N=1))

          res = self._extend_with_top_popular(res, N=N)

          assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
          return res

    def get_top_items_for_user(self, user):
      """Для каждого пользователя выделяет топ самых покупаемых
        товаров, стоимость которых выше mprice_theashold
      """
      if user in self.top_items_for_user.index:
        if type(user) == 'list':
          return self.top_items_for_user.loc[user, 'item_id'].values.tolist()
        else:
          return self.top_items_for_user.loc[user, 'item_id'].tolist()
      else:
        return [] # Если по этому пользователю не было информации

    def get_all_top_items(self):
      """Возвращает топ из товаров, которые покупает наибольшее число уникальных пользователей
      (стоимость товаров больше mprice_theashold)
      """ 
      return self.all_top_items

    def get_top_items_in_users_basket(self, user):
      """Для каждого пользователя выделяет топ самых покупаемых
        товаров (товары наиболее часто встречаемые к корзинах пользователя), стоимость которых выше mprice_theashold
      """
      if user in self.top_items_in_users_basket.index:
        if type(user) == 'list':
          return self.top_items_in_users_basket.loc[user, 'item_id'].values.tolist()
        else:
          return self.top_items_in_users_basket.loc[user, 'item_id'].tolist()
      else:
        return [] # Если по этому пользователю не было информации

# 3. Класс FILTER

In [70]:
import pandas as pd
import numpy as np

class FilterClass:
    """Содержит функции предфильтрации и постфильтрации

    """

    def __init__(self, data, item_features):

        # Датасет, индексами которого являются товары, а столбцами - 
        # средняя цена по входному датасету для каждого товара и sub_commodity_desc для товара
        self.item_info = data.groupby('item_id').agg({'sales_value':'sum','quantity':'sum'})\
                                                                                    .reset_index()
        # mean_price - результат деления суммы всех sales_value для товара на сумму всех quantity
        # для товара, так как товар может за один раз продаться несколько раз
        self.item_info['mean_price'] = self.item_info['sales_value']/\
                                                                  self.item_info['quantity']
        self.item_info = self.item_info.merge\
                (item_features[['item_id', 'sub_commodity_desc']], how='left', on='item_id')

        self.item_info.index = self.item_info['item_id']
        self.item_info.drop(columns=['sales_value','quantity','item_id'], inplace=True)

        # Датасет, индексами которого являются пользователи, а столбцом - список всех
        # уникальных товаров, которые купил пользователь
        self.users_purchases = data.groupby('user_id')['item_id'].unique().reset_index()
        self.users_purchases.index = self.users_purchases['user_id']
        self.users_purchases.drop(columns = 'user_id', inplace=True)


    def get_mean_prices_from_items(self, recommended_list):
    # Функция возвращает средние цены на товары из списка recommended_list
    # item_info создается при обучении 1 раз для экономии ресурсов
      if type(recommended_list) == 'list':
        return self.item_info.loc[recommended_list, 'mean_price'].values.tolist()
      else:
        return self.item_info.loc[recommended_list, 'mean_price'].tolist()
    
    def get_scd_from_items(self, recommended_list):
    # Функция возвращает sub_commodity_desc для товаров из списка recommended_list
    # item_info создается при обучении 1 раз для экономии ресурсов
      if type(recommended_list) == 'list':
        return self.item_info.loc[recommended_list, 'sub_commodity_desc'].tolist()
      else:
        return self.item_info.loc[recommended_list, 'sub_commodity_desc']

    # Фильтрация в соответствии с бизнес логикой

    def postfilter(self, user_id,	lgbm_candidates,	own_recommender,	all_top_items,	top_items_in_users_basket,	top_items_for_user):
      res = [] 
      com = []
      flag =0
      item = 0
    #1
      for item in top_items_in_users_basket[:55] + own_recommender[:65]:
        if self.get_mean_prices_from_items(item) > 7 :
          res.append(item)
          com.append(self.get_scd_from_items(item))
          if user_id in self.users_purchases.index and item in self.users_purchases.loc[user_id, 'item_id']:
            flag += 1 
          break

      if len(res) == 0:
        for item in lgbm_candidates[:50]+own_recommender[:50] + top_items_for_user[:50]:
          if self.get_mean_prices_from_items(item) > 7:
            res.append(item)
            com.append(self.get_scd_from_items(item))
            if user_id in self.users_purchases.index and item in self.users_purchases.loc[user_id, 'item_id']:
              flag += 1 
            break
      #assert len(res) == 1
        
    # 2    
      for item in top_items_in_users_basket[:75] + own_recommender[:60] + all_top_items[:60] + lgbm_candidates[:60]:
        if (item not in res) and (self.get_scd_from_items(item) not in com):
          res.append(item)
          com.append(self.get_scd_from_items(item))
          if user_id in self.users_purchases.index and item in self.users_purchases.loc[user_id, 'item_id']:
            flag += 1 
          break

      if len(res) != 2:
        for item in own_recommender[:200]+lgbm_candidates[:100]+top_items_for_user[:100]:
          if (item not in res) and (self.get_scd_from_items(item) not in com):
            res.append(item)
            com.append(self.get_scd_from_items(item))
            if user_id in self.users_purchases.index and item in self.users_purchases.loc[user_id, 'item_id']:
              flag += 1 
            break
 
    # 3    
      for item in top_items_in_users_basket[:80]+own_recommender[:80]+all_top_items[:80]+lgbm_candidates[:80]:
        if (item not in res) and (self.get_scd_from_items(item) not in com): 
          res.append(item)
          com.append(self.get_scd_from_items(item))
          if user_id in self.users_purchases.index and item in self.users_purchases.loc[user_id, 'item_id']:
            flag += 1 
          break

      if len(res) != 3:
        for item in all_top_items[:100]+lgbm_candidates[:100]+own_recommender[:100]+top_items_for_user[:100]:
          if (item not in res) and (self.get_scd_from_items(item) not in com):
            res.append(item)
            com.append(self.get_scd_from_items(item))
            if user_id in self.users_purchases.index and item in self.users_purchases.loc[user_id, 'item_id']:
              flag += 1 
            break
    # 4    
      for item in top_items_in_users_basket[:100]+all_top_items[:80]+lgbm_candidates[:80]+own_recommender[:80]:
        if self.get_mean_prices_from_items(item) < 1.35 and  (item not in res) and (self.get_scd_from_items(item) not in com):
          if user_id in self.users_purchases.index and item in self.users_purchases.loc[user_id, 'item_id']:
            if flag < 3:
              flag +=1
              res.append(item)
              com.append(self.get_scd_from_items(item))
              break
          else:
            res.append(item)
            com.append(self.get_scd_from_items(item))
            break

      if len(res) != 4:
        for item in all_top_items+lgbm_candidates+own_recommender :
          if self.get_mean_prices_from_items(item) < 1.2  and  (item not in res) and (self.get_scd_from_items(item) not in com):
            if user_id in self.users_purchases.index and item in self.users_purchases.loc[user_id, 'item_id']:
              if flag < 3:
                flag +=1
                res.append(item)
                com.append(self.get_scd_from_items(item))
                break
              else:
                res.append(item)
                com.append(self.get_scd_from_items(item))
                break 
    # 5   
      for item in all_top_items[:200]+lgbm_candidates[:100]+own_recommender[:100]:
        if self.get_mean_prices_from_items(item) <1.15 and  (item not in res) and (self.get_scd_from_items(item) not in com): 
          if user_id in self.users_purchases.index and item in self.users_purchases.loc[user_id, 'item_id']:
            if flag < 3:
              flag +=1
              res.append(item)
              com.append(self.get_scd_from_items(item))
              break
            else:
              res.append(item)
              com.append(self.get_scd_from_items(item))
              break

      if len(res) != 5:
        for item in all_top_items+own_recommender+lgbm_candidates:
          if self.get_mean_prices_from_items(item) <1.2  and  (item not in res) and (self.get_scd_from_items(item) not in com):
            if user_id in self.users_purchases.index and item in self.users_purchases.loc[user_id, 'item_id']:
              if flag < 3:
                flag +=1
                res.append(item)
                com.append(self.get_scd_from_items(item))
                break
              else:
                res.append(item)
                com.append(self.get_scd_from_items(item))
                break 

      #assert len(res) == 5
      return res           

    def postfilter_items(self, user_id,	lgbm_candidates,	own_recommender,	all_top_items,	top_items_in_users_basket,	top_items_for_user):
    # Функция производит постфильтрацию: из списка с N рекомендациями получает
    # список из 5 товаров, в котором 1 или более товаров стоимостью > 7 дол., 2 или более товаров,
    # которые данный пользователь никогда не покупал и все товары принадлежат разным категориям
      result, scd, price, new = [], [], [], []
      #top_popular = np.array(top_popular)
      # На случай если в recommednations не найдется нужного товара,
      # то искать будем в топе популярных
      #recommednations = np.concatenate([recommednations, top_popular])

      # Если пользователь совершал покупки ранее (о нем есть информация в data)
      if user_id in self.users_purchases.index:

        # Ищем в recommednations 2 товара из различных категорий и сохраняем их хар-ки
        flag = 0
        for item in top_items_for_user + top_items_in_users_basket + own_recommender:
          if self.get_scd_from_items(item) not in scd:        
            flag += 1
            result.append(item)
            scd.append(self.get_scd_from_items(item))
            price.append(self.get_mean_prices_from_items(item))
            new.append(item not in self.users_purchases.loc[user_id, 'item_id'])
            if flag == 2:
              break
        assert len(result)  ==  2 # Должно быть 2 из разных категорий

        # Ищем в recommednations 2 не купленных ранее товара из различных категорий
        # и сохраняем их хар-ки
        flag = 0
        for item in all_top_items + lgbm_candidates:
          if (item not in self.users_purchases.loc[user_id, 'item_id']) and\
                      (self.get_scd_from_items(item) not in scd):
            flag += 1
            result.append(item)
            scd.append(self.get_scd_from_items(item))
            price.append(self.get_mean_prices_from_items(item))
            new.append(item not in self.users_purchases.loc[user_id, 'item_id'])
            if flag == 2:
              break
        assert len(result)  == 4 # Должно быть 2 простых и 2 новых

        # Ищем в recommednations 1-й дорогой товар и сохраняем его хар-ки
        for item in top_items_for_user + top_items_in_users_basket + all_top_items + lgbm_candidates + own_recommender:
          if ((self.get_mean_prices_from_items(item) > 7) and (self.get_scd_from_items(item) not in scd)):
            result.append(item)
            scd.append(self.get_scd_from_items(item))
            price.append(self.get_mean_prices_from_items(item))
            new.append(item not in self.users_purchases.loc[user_id, 'item_id'])
            break
        assert len(result)  == 5 # Один дорогой товар должен быть добавлен
        
      # Если пользователь не совершал покупок ранее 
      else:
          
        # Ищем в recommednations 4 товара из различных категорий и сохраняем их хар-ки
        flag = 0
        for item in lgbm_candidates + all_top_items:
          if (self.get_scd_from_items(item) not in scd) and (self.get_mean_prices_from_items(item) > 3) \
          and (self.get_mean_prices_from_items(item) < 6):        
            flag += 1
            result.append(item)
            scd.append(self.get_scd_from_items(item))
            price.append(self.get_mean_prices_from_items(item))
            new.append(True)
            if flag == 4:
              break
        assert len(result)  == 4 # Должно быть добавлено 4 любых товара

        # Ищем в recommednations 1-й дорогой товар и сохраняем его хар-ки
        for item in lgbm_candidates + all_top_items:
          if (self.get_mean_prices_from_items(item) > 7) and (self.get_scd_from_items(item) not in scd):
            result.append(item)
            scd.append(self.get_scd_from_items(item))
            price.append(self.get_mean_prices_from_items(item))
            new.append(True)
            break
        assert len(result)  == 5 # Один дорогой товар + 4 любых

      assert len(result) == 5
      assert max(price) >= 7 # Один товар должен быть дороже 7
      assert len(set(scd)) == 5 # Все товары должны быть из разных категорий
      assert sum(new) >= 2 # Как минимум 2 новых товара

      # Для отладки выводил: 
      # result - список из 5 рекомендованных товаров
      # scd - соответствующий первому списку список sub_comodity_description
      # price - стоимость рекомендованных товаров
      # old - покупался ли товар ранее
      #
      #return result, scd, price, old
      return result

    @staticmethod
    def prefilter_items(data, take_n_popular=5000, item_features=None):
    # Уберем самые популярные товары (их и так купят)
      popularity = data.groupby('item_id')['user_id'].nunique().reset_index() / data['user_id'].nunique()
      popularity.rename(columns={'user_id': 'share_unique_users'}, inplace=True)

      top_popular = popularity[popularity['share_unique_users'] > 0.2].item_id.tolist()
      data = data[~data['item_id'].isin(top_popular)]

    # Уберем самые НЕ популярные товары (их и так НЕ купят)
      top_notpopular = popularity[popularity['share_unique_users'] < 0.02].item_id.tolist()
      data = data[~data['item_id'].isin(top_notpopular)]

    # Уберем товары, которые не продавались в последние weeks_num недель
      last_week = data['week_no'].max()
      weeks_num = 47
      not_valid = data.groupby('item_id')['week_no'].max().reset_index()
      not_valid = not_valid[not_valid['week_no'] <= (last_week - weeks_num)].item_id.tolist()
      data = data[~data['item_id'].isin(not_valid)]

    # Уберем не интересные для рекомендаций категории (department)
      if item_features is not None:
          department_size = pd.DataFrame(item_features. \
                                       groupby('department')['item_id'].nunique(). \
                                       sort_values(ascending=False)).reset_index()

          department_size.columns = ['department', 'n_items']
          rare_departments = department_size[department_size['n_items'] < 150].department.tolist()
          items_in_rare_departments = item_features[
              item_features['department'].isin(rare_departments)].item_id.unique().tolist()

          data = data[~data['item_id'].isin(items_in_rare_departments)]
    
    # Считаем стоимости для товаров
      #data['price'] = data['sales_value'] / (np.maximum(data['quantity'], 1)) # max - чтобы на 0 не поделить

    # Считаем стоимости для товаров как среднюю стоимость по всему датасету
      price = data.groupby('item_id').agg({'sales_value':'sum','quantity':'sum'}).reset_index()
      price['price'] = price['sales_value']/price['quantity']
      price.drop(columns=['sales_value','quantity'], inplace=True)
      data = data.merge(price, how='left', on='item_id')

    # Уберем слишком дешевые товары (на них не заработаем). 1 покупка из рассылок стоит 60 руб.
      data = data[data['price'] > 1]

    # Уберем слишком дорогие товары
      data = data[data['price'] < 7.5]

    # Возьмем топ по популярности
      popularity = data.groupby('item_id')['quantity'].sum().reset_index()
      popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

      top = popularity.sort_values('n_sold', ascending=False).head(take_n_popular).item_id.tolist()

    # Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
      data.loc[~data['item_id'].isin(top), 'item_id'] = 999999

    # Добавить По популярности (> 10 покупок в неделю) 
    # Добавить По популярности (> 10 покупок в неделю)

      return data

# 4. Обучение одноуровневой модели

In [13]:
data_train_lvl_1_2 = pd.concat([data_train_lvl_1,data_train_lvl_2], axis=0)

In [14]:
# Предфильтрация товаров с помощью prefilter_items 
# Фильтр также добавит среднюю цену для каждого товара
n_items_before = data_train_lvl_1_2['item_id'].nunique()
data_train_lvl_1_2 = FilterClass.prefilter_items(data_train_lvl_1_2, item_features=item_features, take_n_popular=5000)
n_items_after = data_train_lvl_1_2['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 86865 to 5001


In [15]:
# Это оптимальная обработка, если pivot_table строится на основе price
# Добавим признак sales_value(t) = (e^(-max_week_no + week_no)/T)*price
# T - коэффициент затухания (чем больше, тем оно медленнее)
T = 25
data_train_lvl_1_2['price(t)'] = (np.exp((-data_train_lvl_1_2['week_no'].max()+\
                                                  data_train_lvl_1_2['week_no'])/T))*\
                                                  data_train_lvl_1_2['price']

In [16]:
# Обучение модели
single_layer_model = MainRecommender(data_train_lvl_1_2, item_features,
                                     weighting='bm25_weight', 
                                     model_type='als',
                                     n_factors=256, 
                                     regularization=0.01, 
                                     iterations=30,
                                     num_threads=16,
                                     random_state=42)

In [17]:
data_train_lvl_1_2['item_id'].nunique()

5001

In [27]:
# Фильтр должен строиться на основе самого первого датасета (data_train_lvl_1_2)
filter = FilterClass(data_train_lvl_1_2, item_features)

# Формируем выходной датасет
result_lvl_2 = data_val_lvl_2.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_2.columns=['user_id', 'actual']

# Получаем результаты (для каждого пользователя 1000 товаров, чтобы потом выполнить постфильтрацию)
#result_lvl_2['candidates'] = result_lvl_2['user_id'].apply(lambda x: single_layer_model.get_als_recommendations(x, N=200))

result_lvl_2['base_rec'] = result_lvl_2['user_id'].apply(lambda x: single_layer_model.get_own_recommendations(x, N=500))
result_lvl_2['als_rec'] = result_lvl_2['user_id'].apply(lambda x: single_layer_model.get_als_recommendations(x, N=500))

# Оставляем ТОП-5 товаров для каждого пользователя
#result_lvl_2['candidates'] = result_lvl_2['candidates'].apply(lambda x: x[:5])

result_lvl_2.head(2)

,user_id,actual,base_rec,als_rec
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[856942, 8293439, 888104, 5577022, 1082269, 92...","[867188, 1082212, 1132911, 995242, 10149640, 8..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1092937, 13842214, 998206, 904435, 885697, 10...","[1106523, 1053690, 1133018, 1137346, 1092026, ..."


In [28]:
# Добавляем стоимость рекомендованных товаров
result_lvl_2['base_rec_price'] = result_lvl_2['base_rec'].apply(lambda x: filter.get_mean_prices_from_items(x))
result_lvl_2['als_rec_prie'] = result_lvl_2['als_rec'].apply(lambda x: filter.get_mean_prices_from_items(x))

In [29]:
result_lvl_2.head(3)

,user_id,actual,base_rec,als_rec,base_rec_price,als_rec_prie
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[856942, 8293439, 888104, 5577022, 1082269, 92...","[867188, 1082212, 1132911, 995242, 10149640, 8...","[2.7796416938110737, 3.866871165644175, 4.9900...","[1.105548523206752, 2.0154750000000035, 2.1802..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1092937, 13842214, 998206, 904435, 885697, 10...","[1106523, 1053690, 1133018, 1137346, 1092026, ...","[2.402512998266887, 1.9900000000000064, 2.0065...","[2.3801175663760463, 1.1639542413123416, 1.199..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[8203834, 972416, 12757544, 1053329, 13003092,...","[866211, 1023720, 1055504, 1127179, 878996, 11...","[5.438435207823961, 6.185000000000001, 3.1883,...","[3.3179786230745076, 2.2964800559832046, 3.121..."


In [30]:
result_lvl_2['als_MP5'] = result_lvl_2.apply(lambda row: money_precision_at_k(row['als_rec'], row['actual'], row['als_rec_prie'], k=5), axis=1)
als = result_lvl_2['als_MP5'].mean()
result_lvl_2['base_MP5'] = result_lvl_2.apply(lambda row: money_precision_at_k(row['base_rec'], row['actual'], row['base_rec_price'], k=5), axis=1)
base = result_lvl_2['base_MP5'].mean()
base, als # Метрики money precision @ 5 до бизнесфильтра

(0.18889313409774325, 0.13204087406135223)

In [31]:
result_lvl_2.drop(columns=['base_rec_price', 'als_rec_prie', 'base_MP5', 'als_MP5'], inplace=True)

In [32]:
# Добавляем собственные покупки
#result_lvl_2['own_recommender'] = result_lvl_2['user_id'].apply(lambda x: single_layer_model.get_own_recommendations(x, 50))
# Добавляем all_top_items
result_lvl_2['all_top_items'] = result_lvl_2['user_id'].apply(lambda x: single_layer_model.get_all_top_items())
# Добавляем  top_items_in_users_basket
result_lvl_2['top_items_in_users_basket'] = result_lvl_2['user_id'].apply(lambda x: single_layer_model.get_top_items_in_users_basket(x))
# Добавляем top_items_for_user
result_lvl_2['top_items_for_user'] = result_lvl_2['user_id'].apply(lambda x: single_layer_model.get_top_items_for_user(x))

In [33]:
result_lvl_2.head(3)

,user_id,actual,base_rec,als_rec,all_top_items,top_items_in_users_basket,top_items_for_user
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[856942, 8293439, 888104, 5577022, 1082269, 92...","[867188, 1082212, 1132911, 995242, 10149640, 8...","[916122, 965267, 854405, 930118, 863447, 83941...","[1082269, 10149640, 888104, 883616, 953561, 82...","[1082269, 10149640, 888104, 883616, 953561, 82..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1092937, 13842214, 998206, 904435, 885697, 10...","[1106523, 1053690, 1133018, 1137346, 1092026, ...","[916122, 965267, 854405, 930118, 863447, 83941...","[854405, 847573, 12263692, 948953, 1056005, 11...","[847573, 12263692, 854405, 1056005, 948650, 94..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[8203834, 972416, 12757544, 1053329, 13003092,...","[866211, 1023720, 1055504, 1127179, 878996, 11...","[916122, 965267, 854405, 930118, 863447, 83941...","[863447, 13003092, 965267, 1098844, 8203834, 9...","[863447, 13003092, 1098844, 965267, 8203834, 9..."


In [71]:
# Фильтр должен строиться на основе самого первого датасета (data_train_lvl_1_2)
filter = FilterClass(data_train_lvl_1_2, item_features)

In [53]:
# На основе полученного датасета выполняем постфильтрацию
result_lvl_2['result'] = result_lvl_2.apply(lambda row: filter.postfilter(
    user_id = row['user_id'],
    lgbm_candidates = row['als_rec'],
    own_recommender = row['base_rec'],
    all_top_items = row['all_top_items'],
    top_items_in_users_basket = row['top_items_in_users_basket'],
    top_items_for_user = row['top_items_for_user']
), axis=1)

In [54]:
# Добавляем стоимость рекомендованных товаров
result_lvl_2['prices_recommended'] = result_lvl_2['result'].\
                                  apply(lambda x: filter.get_mean_prices_from_items(x))

In [55]:
# Вычислим длины всех предложений
result_lvl_2['row_len'] = result_lvl_2['result'].apply(lambda x: len(x))

print('Кол-во рекомендаций с длиной != 5 ', result_lvl_2[result_lvl_2['row_len'] != 5].shape[0])

Кол-во рекомендаций с длиной != 5  35


In [56]:
result_lvl_2 = result_lvl_2.drop(columns = ['row_len'])
result_lvl_2.head(2)

,user_id,actual,base_rec,als_rec,all_top_items,top_items_in_users_basket,top_items_for_user,result,prices_recommended
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[856942, 8293439, 888104, 5577022, 1082269, 92...","[867188, 1082212, 1132911, 995242, 10149640, 8...","[916122, 965267, 854405, 930118, 863447, 83941...","[1082269, 10149640, 888104, 883616, 953561, 82...","[1082269, 10149640, 888104, 883616, 953561, 82...","[1082269, 856942, 10149640, 867188, 958046]","[4.08126315789474, 2.7796416938110737, 6.54088..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1092937, 13842214, 998206, 904435, 885697, 10...","[1106523, 1053690, 1133018, 1137346, 1092026, ...","[916122, 965267, 854405, 930118, 863447, 83941...","[854405, 847573, 12263692, 948953, 1056005, 11...","[847573, 12263692, 854405, 1056005, 948650, 94...","[854405, 1092937, 847573, 13671759, 1092026]","[6.469165120593573, 2.402512998266887, 5.06997..."


In [57]:
result_lvl_2.apply(lambda row: precision_at_k(row['result'], row['actual'], k=5), axis=1).mean()

0.1275710088148863

In [58]:
result_lvl_2.apply(lambda row: money_precision_at_k(row['result'], row['actual'], row['prices_recommended'], k=5), axis=1).mean()

0.11859978456786814

# 5. Получение результатов

In [61]:
!cp /content/drive/'My Drive'/test.csv /content

In [62]:
test_ds = pd.read_csv('test.csv')

In [64]:
test_ds['base_rec'] = test_ds['user_id'].apply(lambda x: single_layer_model.get_own_recommendations(x, N=500))
test_ds['als_rec'] = test_ds['user_id'].apply(lambda x: single_layer_model.get_als_recommendations(x, N=500))
test_ds['all_top_items'] = test_ds['user_id'].apply(lambda x: single_layer_model.get_all_top_items())
test_ds['top_items_in_users_basket'] = test_ds['user_id'].apply(lambda x: single_layer_model.get_top_items_in_users_basket(x))
test_ds['top_items_for_user'] = test_ds['user_id'].apply(lambda x: single_layer_model.get_top_items_for_user(x))

In [72]:
# На основе полученного датасета выполняем постфильтрацию
test_ds['result'] = test_ds.apply(lambda row: filter.postfilter(
    user_id = row['user_id'],
    lgbm_candidates = row['als_rec'],
    own_recommender = row['base_rec'],
    all_top_items = row['all_top_items'],
    top_items_in_users_basket = row['top_items_in_users_basket'],
    top_items_for_user = row['top_items_for_user']
), axis=1)

In [74]:
test_ds = test_ds[['user_id', 'result']]

In [77]:
test_ds.to_csv('test.csv')